In [2]:
## Imports
import pandas as pd
from tqdm import tqdm
from eod import EodHistoricalData
from datetime import datetime, timedelta
from pytz import timezone
from pathlib import Path
import io
import json
import requests
from websocket import create_connection
tz = timezone('EST')
datetime.now(tz)
api_key = '636bdae5d4c4f6.65605075'
conn = EodHistoricalData(api_key)
index = 'GSPC.INDX'

In [7]:
requests.get(f'https://eodhistoricaldata.com/api/fundamentals/AAPL.US?api_token={api_key}&fmt=json').json()['Financials'][]

,currency_symbol,quarterly,yearly
2022-12-31,USD,"{'date': '2022-12-31', 'filing_date': '2023-02...",NaN
2022-09-30,USD,"{'date': '2022-09-30', 'filing_date': '2022-10...","{'date': '2022-09-30', 'filing_date': '2022-10..."
2022-06-30,USD,"{'date': '2022-06-30', 'filing_date': '2022-07...",NaN
2022-03-31,USD,"{'date': '2022-03-31', 'filing_date': '2022-04...",NaN
2021-12-31,USD,"{'date': '2021-12-31', 'filing_date': '2022-01...",NaN
...,...,...,...
1986-09-30,USD,"{'date': '1986-09-30', 'filing_date': '1986-09...","{'date': '1986-09-30', 'filing_date': '1986-09..."
1986-06-30,USD,"{'date': '1986-06-30', 'filing_date': '1986-06...",NaN
1986-03-31,USD,"{'date': '1986-03-31', 'filing_date': '1986-03...",NaN
1985-12-31,USD,"{'date': '1985-12-31', 'filing_date': '1985-12...",NaN


In [8]:
conn = EodHistoricalData(api_key)

In [28]:
components = pd.DataFrame(conn.get_fundamental_equity(index)['Components']).transpose()
components = components.sort_values(by='Code').reset_index()[['Code', 'Name']]

In [29]:
components

,Code,Name
0,A,Agilent Technologies Inc
1,AAL,American Airlines Group
2,AAP,Advance Auto Parts Inc
3,AAPL,Apple Inc
4,ABBV,AbbVie Inc
...,...,...
498,YUM,Yum! Brands Inc
499,ZBH,Zimmer Biomet Holdings Inc
500,ZBRA,Zebra Technologies Corporation
501,ZION,Zions Bancorporation


In [147]:
financial = pd.DataFrame()
for symbol in tqdm(components.Code):
    merged_table = pd.DataFrame()
    sp500_fundamental_bs = pd.DataFrame(conn.get_fundamental_equity(symbol, filter_='Financials::Balance_Sheet::quarterly')).transpose().drop(['filing_date', 'currency_symbol'], axis=1)
    sp500_fundamental_is = pd.DataFrame(conn.get_fundamental_equity(symbol, filter_='Financials::Income_Statement::quarterly')).transpose().drop(['filing_date', 'currency_symbol'], axis=1)
    sp500_fundamental_cf = pd.DataFrame(conn.get_fundamental_equity(symbol, filter_='Financials::Cash_Flow::quarterly')).transpose().drop(['filing_date', 'currency_symbol'], axis=1)
    merged_table = sp500_fundamental_bs.merge(sp500_fundamental_is,how = 'left', on = 'date')
    total_financial_table = merged_table.merge(sp500_fundamental_cf,how = 'left', on = 'date')
    total_financial_table['Code'] = symbol
    total_financial_table = total_financial_table.sort_values(by=['date'])
    financial = pd.concat([financial,total_financial_table],axis = 0)

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [05:18<00:00,  1.58it/s]


In [49]:
financial.to_csv(Path('D:/bpgs/data/financial_data.csv'),index=False)

In [10]:
conn = EodHistoricalData(api_key)

In [43]:
running, count,times = '', 0,0
price_list = []

for symbol in tqdm(components.Code):

    if count == 151:
        price_newclose = conn.get_prices_live(symbol, s=running)
        price_newclose = pd.DataFrame(price_newclose)
        price_list.append(price_newclose)
        running = ''
        count = 0
        times += 1
    elif count < 151:
        count += 1
        running += symbol + ','
   

price_newclose = conn.get_prices_live(symbol, s=running)
price_newclose = pd.DataFrame(price_newclose)
price_list.append(price_newclose)
        

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:07<00:00, 69.63it/s]


In [44]:
pd.concat(price_list)

,code,timestamp,gmtoffset,open,high,low,close,volume,previousClose,change,change_p
0,EBAY.US,1680021660,0,42.72,42.9650,42.6650,42.7450,846818,42.60,0.1450,0.3404
1,A.US,1680021720,0,132.89,133.2900,132.0108,132.9686,291904,133.07,-0.1014,-0.0762
2,AAL.US,1680021720,0,13.85,14.0700,13.8500,13.9350,5926443,13.94,-0.0050,-0.0359
3,AAP.US,1680021660,0,114.86,116.6540,114.8250,116.0800,420685,114.74,1.3400,1.1679
4,AAPL.US,1680021720,0,157.91,158.1277,156.0800,156.6400,22152803,158.28,-1.6400,-1.0361
...,...,...,...,...,...,...,...,...,...,...,...
42,XYL.US,1680021660,0,99.38,100.7600,99.3000,100.1200,545659,99.41,0.7100,0.7142
43,YUM.US,1680021720,0,128.18,128.9700,128.0800,128.0900,316059,128.29,-0.2000,-0.1559
44,ZBH.US,1680021660,0,125.06,127.1250,124.9100,126.6700,268092,126.00,0.6700,0.5317
45,ZBRA.US,1680021660,0,287.76,289.5100,286.5250,287.7600,54551,288.86,-1.1000,-0.3808


In [56]:
mani = pd.read_excel('D:/实习/bpgs/data_fundamental/CompanyFundamentals_wIDs.xlsx',sheet_name='CompanyFundamentals_wIDs')

In [65]:
header_row = mani.iloc[0]
verify = pd.DataFrame(mani.values[1:], columns=header_row)

In [69]:
verify = verify.drop_duplicates().reset_index()

In [79]:
verify_new = verify.drop(verify.columns[0:2],axis = 1)

In [138]:
verify_new['Balance Sheet Period End Date']

0       1999-07-31
1       1999-10-31
2       2000-01-31
3       2000-04-30
4       2000-07-31
           ...    
82120          NaT
82121          NaT
82122          NaT
82123          NaT
82124          NaT
Name: Balance Sheet Period End Date, Length: 82125, dtype: datetime64[ns]

In [121]:
tick = []
for symbol in tqdm(components.Code):
    tickers = pd.DataFrame(requests.get(f"https://eodhistoricaldata.com/api/search/{symbol}?api_token={api_key}&limit=1").json())
    tick.append(tickers)

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [02:38<00:00,  3.18it/s]


In [126]:
df = pd.concat(tick)

,Code,Exchange,Name,Type,Country,Currency,ISIN,previousClose,previousCloseDate
0,A,US,Agilent Technologies Inc,Common Stock,USA,USD,US00846U1016,133.07,2023-03-27
0,AAL,LSE,Anglo American PLC,Common Stock,UK,GBX,GB00B1XZS820,2567.50,2023-03-28
0,AAP,US,Advance Auto Parts Inc,Common Stock,USA,USD,US00751Y1064,114.74,2023-03-27
0,AAPL,US,Apple Inc,Common Stock,USA,USD,US0378331005,158.28,2023-03-27
0,ABBV,US,AbbVie Inc,Common Stock,USA,USD,US00287Y1091,157.54,2023-03-27
...,...,...,...,...,...,...,...,...,...
0,YUM,US,Yum! Brands Inc,Common Stock,USA,USD,US9884981013,128.29,2023-03-27
0,ZBH,US,Zimmer Biomet Holdings Inc,Common Stock,USA,USD,US98956P1021,126.00,2023-03-27
0,ZBRA,US,Zebra Technologies Corporation,Common Stock,USA,USD,US9892071054,288.86,2023-03-27
0,ZION,US,Zions Bancorporation,Common Stock,USA,USD,US9897011071,29.33,2023-03-27


In [149]:
fundamental_new = pd.merge(financial,df[['Code','ISIN']],how = 'inner',on='Code')

In [162]:
fundamental_new['date'] = pd.to_datetime(fundamental_new['date'])

In [163]:
fundamental_new[['date','Code','ISIN','totalAssets']]

,date,Code,ISIN,totalAssets
0,1999-01-31,A,US00846U1016,None
1,1999-04-30,A,US00846U1016,None
2,1999-07-31,A,US00846U1016,None
3,1999-10-31,A,US00846U1016,5444000000.00
4,2000-01-31,A,US00846U1016,7107000000.00
...,...,...,...,...
60340,2021-12-31,ZTS,US98978V1035,13900000000.00
60341,2022-03-31,ZTS,US98978V1035,13860000000.00
60342,2022-06-30,ZTS,US98978V1035,13770000000.00
60343,2022-09-30,ZTS,US98978V1035,13674000000.00


In [156]:
verify_new = verify_new.rename(columns={'Balance Sheet Period End Date':'date'})

In [169]:
verify_new.columns

Index(['Instrument', 'Qtrs Same Ticker', 'Company Name', 'CUSIP', 'ISIN',
       'OAPermID', 'SEDOL', 'Entry Date', 'Exit Date', 'Total Duration',
       'date', 'Cash and Equivalents', 'Accounts Receivable (CF)',
       'Total Receivables, Net', 'Total Inventory', 'Total Current Assets',
       'Other Current Assets', 'Goodwill, Net', 'Intangibles, Net',
       'Other Long Term Assets, Total', 'Total Assets, Reported',
       'Notes Payable/Short Term Debt', 'Total Current Liabilities',
       'Total Long Term Debt', 'Total Liabilities', 'Other Liabilities',
       'Total Liabilities.1', 'Common Stock, Total',
       'Retained Earnings (Accumulated Deficit)', 'Unrealized Gain (Loss)',
       'Total Equity', 'Total Revenue', 'Chg in Rev',
       'Research And Development',
       'Depreciation, Depletion And Amortization, Discrete',
       'Interest Expense', 'Total Operating Expense', 'Operating Income',
       'Net Income Before Taxes', 'Net Income Incl Extra Before Distributions',
 

In [188]:
fundamental_new.columns[100:130]

Index(['changeInCash', 'beginPeriodCashFlow', 'endPeriodCashFlow',
       'totalCashFromOperatingActivities', 'issuanceOfCapitalStock',
       'depreciation', 'otherCashflowsFromInvestingActivities',
       'dividendsPaid', 'changeToInventory', 'changeToAccountReceivables',
       'salePurchaseOfStock', 'otherCashflowsFromFinancingActivities',
       'changeToNetincome', 'capitalExpenditures', 'changeReceivables',
       'cashFlowsOtherOperating', 'exchangeRateChanges',
       'cashAndCashEquivalentsChanges', 'changeInWorkingCapital',
       'stockBasedCompensation', 'otherNonCashItems', 'freeCashFlow', 'Code',
       'ISIN'],
      dtype='object')

In [191]:
fundamental_new.columns[0:100]

Index(['date', 'totalAssets', 'intangibleAssets', 'earningAssets',
       'otherCurrentAssets', 'totalLiab', 'totalStockholderEquity',
       'deferredLongTermLiab', 'otherCurrentLiab', 'commonStock',
       'capitalStock', 'retainedEarnings', 'otherLiab', 'goodWill',
       'otherAssets', 'cash', 'cashAndEquivalents', 'totalCurrentLiabilities',
       'currentDeferredRevenue', 'netDebt', 'shortTermDebt',
       'shortLongTermDebt', 'shortLongTermDebtTotal', 'otherStockholderEquity',
       'propertyPlantEquipment', 'totalCurrentAssets', 'longTermInvestments',
       'netTangibleAssets', 'shortTermInvestments', 'netReceivables',
       'longTermDebt', 'inventory', 'accountsPayable', 'totalPermanentEquity',
       'noncontrollingInterestInConsolidatedEntity',
       'temporaryEquityRedeemableNoncontrollingInterests',
       'accumulatedOtherComprehensiveIncome', 'additionalPaidInCapital',
       'commonStockTotalEquity', 'preferredStockTotalEquity',
       'retainedEarningsTotalEquity',

In [316]:
df = fundamental_new[['date','Code','ISIN','cashAndEquivalents','netReceivables','inventory','totalCurrentAssets',
                              'otherCurrentAssets','goodWill','intangibleAssets','otherAssets','totalAssets','shortTermDebt',
                              'totalCurrentLiabilities','longTermDebt','totalLiab','otherLiab','commonStock','retainedEarnings',
                              'totalStockholderEquity','totalRevenue','researchDevelopment','depreciationAndAmortization',
                              'interestExpense','totalOperatingExpenses','operatingIncome','incomeBeforeTax','netIncome_x',
                              'totalCashFromOperatingActivities','capitalExpenditures','totalCashflowsFromInvestingActivities',
                              'dividendsPaid','issuanceOfCapitalStock','totalCashFromFinancingActivities','commonStockSharesOutstanding'.
                         ]]

In [318]:
df.iloc[:,3:] = df.iloc[:,3:].astype(float)

C:\Users\HP\AppData\Local\Temp/ipykernel_11304/1218071708.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[:,3:] = df.iloc[:,3:].astype(float)
C:\Users\HP\AppData\Local\Temp/ipykernel_11304/1218071708.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,3:] = df.iloc[:,3:].astype(float)


In [367]:
date_list = [date. strftime('%Y-%m') for date in df.date]

In [368]:
df['date']=date_list

C:\Users\HP\AppData\Local\Temp/ipykernel_11304/881863184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date']=date_list


In [370]:
refin = verify_new[['Instrument','ISIN','date','Cash and Equivalents', 'Accounts Receivable (CF)','Total Receivables, Net',
                         'Total Inventory', 'Total Current Assets','Other Current Assets', 'Goodwill, Net', 'Intangibles, Net',
                         'Other Long Term Assets, Total', 'Total Assets, Reported','Notes Payable/Short Term Debt', 
                         'Total Current Liabilities','Total Long Term Debt', 'Total Liabilities', 'Other Liabilities',
                         'Total Liabilities.1', 'Common Stock, Total','Retained Earnings (Accumulated Deficit)', 
                         'Unrealized Gain (Loss)','Total Equity','Total Revenue', 'Chg in Rev','Research And Development',
                         'Depreciation, Depletion And Amortization, Discrete','Interest Expense', 'Total Operating Expense', 
                         'Operating Income','Net Income Before Taxes', 'Net Income Incl Extra Before Distributions',
                         'Basic EPS Including Extraordinary Items','Cash from Operating Activities, Cumulative',
                         'Capital Expenditures, Cumulative','Cash from Investing Activities, Cumulative',
                         'Cash Dividends Paid - Common','Issuance (Retirement) of Stock, Net, Cumulative',
                         'Cash from Financing Activities, Cumulative']]

In [374]:
refin_list = []
for date in refin.date:
    try:
        refin_list.append(date.strftime('%Y-%m'))
    except Exception:
        refin_list.append(np.nan)

In [375]:
refin['date'] = refin_list

C:\Users\HP\AppData\Local\Temp/ipykernel_11304/1783786663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refin['date'] = refin_list


In [383]:
a = pd.merge(refin,df,on =['ISIN','date'])

In [382]:
a.to_csv('D:/实习/bpgs/data_fundamental/balance_sheet.csv',index=False)

In [359]:
df1 = verify_new[['Instrument','ISIN','date','Cash and Equivalents']]

In [386]:
a[['Instrument','date','Cash and Equivalents','cashAndEquivalents']]

,Instrument,date,Cash and Equivalents,cashAndEquivalents
0,A.N,1999-07,NaN,NaN
1,A.N,1999-10,0,NaN
2,A.N,2000-01,1368000000,NaN
3,A.N,2000-04,978000000,NaN
4,A.N,2000-07,703000000,NaN
...,...,...,...,...
38997,ZTS.N,2021-06,3657000000,3.657000e+09
38998,ZTS.N,2021-09,3272000000,3.272000e+09
38999,ZTS.N,2021-12,3482000000,3.482000e+09
39000,ZTS.N,2022-03,3132000000,3.132000e+09


In [358]:
print(df.iloc[:,:4][df.iloc[:,:4]['Code']=='AAPL'])
print(df1[df1['Instrument']=='AAPL.OQ'])

          date  Code          ISIN  cashAndEquivalents
338 1985-09-30  AAPL  US0378331005        3.370000e+08
339 1985-12-31  AAPL  US0378331005                 NaN
340 1986-03-31  AAPL  US0378331005                 NaN
341 1986-06-30  AAPL  US0378331005                 NaN
342 1986-09-30  AAPL  US0378331005        5.762000e+08
..         ...   ...           ...                 ...
483 2021-12-31  AAPL  US0378331005        3.711900e+10
484 2022-03-31  AAPL  US0378331005        2.809800e+10
485 2022-06-30  AAPL  US0378331005        2.750200e+10
486 2022-09-30  AAPL  US0378331005        2.364600e+10
487 2022-12-31  AAPL  US0378331005        2.053500e+10

[150 rows x 4 columns]
0            A.N
1            A.N
2            A.N
3            A.N
4            A.N
          ...   
82120    ZION.OQ
82121      ZTS.N
82122      ZTS.N
82123      ZTS.N
82124      ZTS.N
Name: Instrument, Length: 82125, dtype: object


In [330]:
a[a['Code'] == 'AAPL']

,Instrument,ISIN,date_x,Cash and Equivalents,date_y,Code,cashAndEquivalents
18179,AAPL.OQ,US0378331005,1997-03-28,1273000000,1985-09-30,AAPL,3.370000e+08
18180,AAPL.OQ,US0378331005,1997-03-28,1273000000,1985-12-31,AAPL,NaN
18181,AAPL.OQ,US0378331005,1997-03-28,1273000000,1986-03-31,AAPL,NaN
18182,AAPL.OQ,US0378331005,1997-03-28,1273000000,1986-06-30,AAPL,NaN
18183,AAPL.OQ,US0378331005,1997-03-28,1273000000,1986-09-30,AAPL,5.762000e+08
...,...,...,...,...,...,...,...
33624,AAPL.OQ,US0378331005,NaT,NaN,2021-12-31,AAPL,3.711900e+10
33625,AAPL.OQ,US0378331005,NaT,NaN,2022-03-31,AAPL,2.809800e+10
33626,AAPL.OQ,US0378331005,NaT,NaN,2022-06-30,AAPL,2.750200e+10
33627,AAPL.OQ,US0378331005,NaT,NaN,2022-09-30,AAPL,2.364600e+10


In [463]:
s_list = [] 
for symbol in tqdm(components.Code):
    s = pd.DataFrame(conn.get_market_cap(symbol)).transpose()
    s['date'] = pd.to_datetime(s['date'])
    s = s.set_index(['date'])
    s = s.resample('M').last()
    s = s.reset_index()
    s['date'] = [date.strftime('%Y-%m') for date in s.date]
    s['Code'] = symbol
    s_list.append(s)

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [01:54<00:00,  4.39it/s]


In [464]:
s = pd.concat(s_list)
s

,date,value,Code
0,2018-03,21302804200,A
1,2018-04,21168280000,A
2,2018-05,19789037400,A
3,2018-06,19830631200,A
4,2018-07,21066760000,A
...,...,...,...
56,2022-11,71840338100,ZTS
57,2022-12,67035135800,ZTS
58,2023-01,76764878600,ZTS
59,2023-02,77121448500,ZTS


In [465]:
final = a.merge(s,how='left',on=['date','Code'])

In [467]:
final.to_csv('D:/实习/bpgs/data_fundamental/balance_sheet.csv',index=False)